In [14]:
import qiskit
print(qiskit.__version__)

1.0.2


In [15]:
import json

In [16]:
def read_data(data_file):
    file = open(data_file)
    data = json.load(file)
    return data

In [17]:
data = read_data("sample.json")

# data = {
#     "0": {"connected_nodes": [{"node": "5", "weight": 5}, {"node": "6", "weight": 3}], "quantity": 5, "quality": 3},
#     "1": {"connected_nodes": [{"node": "5", "weight": 7}, {"node": "7", "weight": 4}], "quantity": 4, "quality": 2},
#     "2": {"connected_nodes": [{"node": "6", "weight": 2}, {"node": "8", "weight": 6}], "quantity": 3, "quality": 4},
#     "3": {"connected_nodes": [{"node": "7", "weight": 1}, {"node": "9", "weight": 5}], "quantity": 6, "quality": 5},
#     "4": {"connected_nodes": [{"node": "8", "weight": 8}, {"node": "9", "weight": 2}], "quantity": 2, "quality": 1},
#     "5": {"connected_nodes": [{"node": "0", "weight": 5}, {"node": "1", "weight": 7}], "quantity": 7, "quality": 4},
#     "6": {"connected_nodes": [{"node": "0", "weight": 3}, {"node": "2", "weight": 2}], "quantity": 8, "quality": 3},
#     "7": {"connected_nodes": [{"node": "1", "weight": 4}, {"node": "3", "weight": 1}], "quantity": 9, "quality": 2},
#     "8": {"connected_nodes": [{"node": "2", "weight": 6}, {"node": "4", "weight": 8}], "quantity": 4, "quality": 5},
#     "9": {"connected_nodes": [{"node": "3", "weight": 5}, {"node": "4", "weight": 2}], "quantity": 6, "quality": 1}
# }

In [48]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Pauli
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer import Aer
from qiskit import QuantumCircuit

# Define the QUBO coefficients
def get_qubo_coefficients(data):
    num_nodes = len(data)
    nodes = data.keys()
    coefficients = {}
    
    for i in range(len(data)):
        for j in range(len(data)):
            coefficients[(str(i), str(i))] = 0
    
    # Penalty term coefficients
    alpha = 1000  # Penalty coefficient for water supply constraints
    beta = 1000  # Penalty coefficient for water demand constraints
    gamma = 1000  # Penalty coefficient for water quality constraints
    delta = 1000  # Penalty coefficient for flow conservation constraints

    # Original objective function coefficients
    # for i in range(num_nodes):
    #     for j in range(len(data[str(i)]['connected_nodes'])):
    #         node_j = int(data[str(i)]['connected_nodes'][j]['node'])
    #         weight = data[str(i)]['connected_nodes'][j]['weight']
    #         coefficients[(str(i), str(node_j))] = weight
    
    count = 1
    
    for node in nodes:
        
        connected_nodes = data[node]['connected_nodes']
        
        if data[node]['type'] == 'Source':
                coefficients[(node, node)] += alpha
                
        for conn_node in connected_nodes:
            n = str(conn_node['node'])
            w = conn_node['weight']
            coefficients[(node, n)] = w
            
            # Water supply constraints
            if data[node]['type'] == 'Source':
                coefficients[(node, n)] -= 2*alpha*data[n]['quantity']
                
            # Water demand constraints
            if data[node]['type'] == 'Destination':
                coefficients[(node, n)] -= 2*beta*data[n]['quantity']
                
        count += 1
            
    # # Water supply constraints
    # for i in range(5):  # Assuming nodes 0 to 4 are source nodes
    #     coefficients[(i, i)] += alpha
    #     coefficients[(i, num_nodes)] = -2 * alpha * data[str(i)]['quantity']
        
#     # Water demand constraints
#     for j in range(5, num_nodes):  # Assuming nodes 5 and above are destination nodes
#         for i in range(num_nodes):
#             coefficients[(i, j)] += beta
#         coefficients[(j, num_nodes)] = -2 * beta * data[str(j)]['quantity']

#     # Water quality constraints
#     for j in range(5, num_nodes):  # Assuming nodes 5 and above are destination nodes
#         for i in range(5):  # Assuming nodes 0 to 4 are source nodes
#             coefficients[(i, j)] += gamma * data[str(j)]['quality']
#             coefficients[(i, num_nodes)] -= gamma * data[str(j)]['quality'] * data[str(i)]['quality']

#     # Flow conservation constraints
#     for i in range(num_nodes):
#         for j in range(num_nodes):
#             coefficients[(i, j)] += delta
#         for k in range(num_nodes):
#             coefficients[(k, i)] -= delta

    return coefficients

# Convert QUBO coefficients to Hamiltonian
def get_hamiltonian(coefficients, num_nodes):
#    pauli_list = []
#     for (i, j), coeff in coefficients.items():
#         print(i,j)
#         if i == j:
            
#             pauli_list.append(coeff * SparsePauliOp(Pauli('Z' * i)))
#         else:
#             pauli_list.append(coeff * SparsePauliOp(Pauli('Z' * i)) * SparsePauliOp(Pauli('Z' * j)))

#     hamiltonian = PauliSumOp(pauli_list)
#     return hamiltonian

    pauli_terms = []
    for (u, v), coeff in qubo_coefficients.items():
        
        i = int(u)
        j = int(v)
        
        print('Z'*i)
        
        if i == j:
            pauli_terms.append((coeff, 'Z'*i + 'I'*(num_qubits-i-1)))
        else:
            pauli_terms.append((coeff, 'Z'*min(i, j) + 'Z' + 'I'*(abs(i-j)-1) + 'Z' + 'I'*(num_qubits-max(i, j)-1)))

    hamiltonian = SparsePauliOp(pauli_terms, num_qubits=num_qubits)
    return hamiltonian
    
#     for (i, j), coeff in coefficients.items():
#         print(i,j)
        
#         if j == num_nodes:
#             pauli_list.append((coeff, Pauli('I' * i + 'Z' + 'I' * (num_nodes - i - 1))))
#         else:
#             pauli_list.append((coeff, Pauli('I' * i + 'Z' + 'I' * (j - i - 1) + 'Z' + 'I' * (num_nodes - j - 1))))
            
#     hamiltonian = SparsePauliOp(pauli_list)
#     return hamiltonian


# Solve the QUBO using VQE
def solve_qubo(hamiltonian, num_nodes):
    # Define the ansatz circuit
    ansatz = TwoLocal(num_nodes, ['ry', 'rz'], 'cz', reps=1)

    # Define the classical optimizer
    optimizer = COBYLA(maxiter=1000)

    # Run VQE
    backend = Aer.get_backend('statevector_simulator')
    vqe = VQE(ansatz, optimizer, quantum_instance=backend)
    result = vqe.compute_minimum_eigenvalue(hamiltonian)

    return result.eigenstate

num_nodes = len(data)

# Get QUBO coefficients
coefficients = get_qubo_coefficients(data)

# Convert QUBO to Hamiltonian
hamiltonian = get_hamiltonian(coefficients, num_nodes)

# Solve the QUBO using VQE
optimal_state = solve_qubo(hamiltonian, num_nodes)

# Print the optimal flow
for i in range(num_nodes):
    for j in range(num_nodes):
        pauli_op = SparsePauliOp(['I' * i + 'Z' + 'I' * (j - i - 1) + 'Z' + 'I' * (num_nodes - j - 1)])
        if abs(optimal_state.expectation_value(pauli_op).real) > 0.5:
            print(f'Water flows from node {i} to node {j}')


#print(get_qubo_coefficients(data))


Z



TypeError: __init__() got an unexpected keyword argument 'num_qubits'

In [45]:
print('Z'*5)

ZZZZZ
